In [ ]:
from Environment.custom_env import CustomEnvironment
import json
import torch
import numpy as np
from tqdm._tqdm_notebook import tqdm_notebook

In [ ]:
torch.cuda.is_available()

In [ ]:
f = open('Environment/env_config.json')
env_variables = json.load(f)
f.close

In [ ]:
# f = open('Models/model_config.json')
# model_variables = json.load(f)
# f.close

In [ ]:
# ac_0 = ActorCritic(model_variables)
# ac_1 = ActorCritic(model_variables)
# ac_2 = ActorCritic(model_variables)
# ac_3 = ActorCritic(model_variables)
# ac_4 = ActorCritic(model_variables)

# #agent_id to object mapping
# objects = {
    
#     0 : ac_0,
#     1 : ac_1,
#     2 : ac_2,
#     3 : ac_3,
#     4 : ac_4
# }  

In [ ]:
env = CustomEnvironment(env_config=env_variables)
# env = parallel_env(render_mode="human")
# env.reset(seed=42)

In [ ]:
# parallel_api_test(env, num_cycles=1000)

In [ ]:
# observations, infos = env.reset()
# while env.agents:
#     # print(env.timestep)
#     # this is where you would insert your policy
#     actions = {agent: env.action_space(agent).sample() for agent in env.agents}
    
#     observations, rewards, terminations, truncations, infos = env.step(actions)

# env.close()

In [ ]:
env.agents

In [ ]:
episodes = 100

for i in tqdm_notebook(range(episodes)): #episodes loop"

    curr_obs,infos = env.reset() #reset environment after each episode\n",

    while(env.agents): #until there are any surviving agents 

        actions = {}
        log_probs = {}

        for i in range(env.n_agents): #objects contains all of the models, get the corresponding actions from each policy\n",
            
            if i in env.agents:
                act, log_prob = env.agents_objects[i].brain1.select_action(state = torch.tensor(curr_obs[i], dtype = torch.float32))
                actions[i] = act
                log_probs[i] = log_prob
            else:
                actions[i] = env.num_actions
                log_probs[i] = 69
            
        # print(actions)

        next_obs, rewards, terminations, truncations, infos = env.step(actions)
        
        # print(env.agents_objects)
        # print(curr_obs)
        # print(rewards)
        # print(terminations)
        # print(truncations)
        # print(infos)

        for i in range(env.n_agents):

            if i in env.agents:
                env.agents_objects[i].brain1.push_to_buffer(torch.tensor(curr_obs[i], dtype = torch.float32),actions[i],rewards[i],torch.tensor(next_obs[i], dtype = torch.float32),log_probs[i],terminations[i])
                env.agents_objects[i].brain1.update_weights() 
            
            if i in env.justdie:
                env.justdie[i].brain1.push_to_buffer(torch.tensor(curr_obs[i], dtype = torch.float32),actions[i],rewards[i],torch.tensor(next_obs[i], dtype = torch.float32),log_probs[i],terminations[i])
                env.justdie[i].brain1.update_weights() 
                del env.justdie[i]
                
        
        curr_obs = next_obs
